# Proyecto de Pronóstico de Series de Tiempo: Acciones de Take-Two Interactive (TTWO)
### Problema y Datos
**Objetivo:** Pronosticar el precio de cierre ("Close") de las acciones de Take-Two Interactive Software, Inc. (TTWO) para los próximos 7 días.

**Variable de Interés:** Precio de cierre diario ajustado en USD. Motivación: TTWO es una empresa líder en videojuegos (dueña de Rockstar Games, 2K). Predecir su comportamiento a corto plazo es crucial para la toma de decisiones de inversión y trading algorítmico, especialmente dada la volatilidad del sector tecnológico.

In [43]:
import numpy as np
import pandas as pd
import yfinance as yf
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, mean_absolute_error
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Input
import math
from datetime import timedelta


In [68]:
ticker = 'TTWO'
start_date = '2013-01-01'
end_date = False

# Descarga de datos
df = yf.download(ticker, start=start_date, end=end_date)

# Limpieza básica
df.reset_index(inplace=True)
df.columns = df.columns.get_level_values(0) # Elimina multi-index si existe
df = df[['Date', 'Close']]
df['Date'] = pd.to_datetime(df['Date'])

# Verificación de nulos
print(f"Nulos encontrados: {df.isnull().sum().sum()}")
fig = px.line(df, x='Date', y='Close', title=f'Serie de Tiempo Histórica: {ticker}')
fig.update_layout(template="plotly_dark")
fig.show()

C:\Users\canal\AppData\Local\Temp\ipykernel_11936\890728684.py:6: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed

Nulos encontrados: 0


In [69]:
# LSTM es sensible a la magnitud, usamos MinMaxScaler (0,1)
data = df.filter(['Close']).values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)


In [70]:
# Usamos los 60 días previos para predecir el día 61
window_size = 65
training_data_len = math.ceil(len(scaled_data) * 0.8)

train_data = scaled_data[0:training_data_len, :]
X_train, y_train = [], []

for i in range(window_size, len(train_data)):
    X_train.append(train_data[i-window_size:i, 0])
    y_train.append(train_data[i, 0])

X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

print(f"Datos de entrenamiento preparados. Shape: {X_train.shape}")

Datos de entrenamiento preparados. Shape: (2534, 65, 1)


# Modelado
**Modelo Elegido: LSTM (Long Short-Term Memory)** Se eligió una red neuronal recurrente LSTM porque:

1. Es capaz de aprender dependencias a largo plazo en datos secuenciales, solucionando el problema del desvanecimiento del gradiente de las RNN tradicionales.

2. Es ideal para series de tiempo financieras donde el precio de hoy depende no solo de ayer, sino de tendencias formadas semanas atrás.

**Arquitectura e Hiperparámetros:**

- **Capas LSTM:** 3 capas apiladas de 64 unidades para capturar complejidad.

- **Dropout (0.2):** Para evitar overfitting (sobreajuste), apagando aleatoriamente el 20% de las neuronas durante el entrenamiento.

- **Optimizador:** Adam, estándar por su eficiencia en convergencia.

- **Loss Function:** Mean Squared Error (MSE), penaliza los errores grandes, ideal para regresión.

In [71]:
# Construir el modelo LSTM
model = Sequential()
model.add(Input(shape=(X_train.shape[1], 1)))
model.add(LSTM(units=64, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=64, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=64, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu')) 
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_8 (LSTM)                   │ (None, 65, 64)         │        16,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 65, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_9 (LSTM)                   │ (None, 65, 64)         │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 65, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_10 (LSTM)                  │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 85,057 (332.25 KB)

 Trainable params: 85,057 (332.25 KB)

 Non-trainable params: 0 (0.00 B)

In [72]:
# Entrenamiento del modelo
history = model.fit(X_train, y_train, batch_size=32, epochs=45, validation_split=0.1, verbose=1)

Epoch 1/45


72/72 ━━━━━━━━━━━━━━━━━━━━ 8s 53ms/step - loss: 0.0070 - val_loss: 9.1362e-04
Epoch 2/45
72/72 ━━━━━━━━━━━━━━━━━━━━ 4s 51ms/step - loss: 0.0015 - val_loss: 0.0017
Epoch 3/45
72/72 ━━━━━━━━━━━━━━━━━━━━ 3s 46ms/step - loss: 0.0015 - val_loss: 0.0012
Epoch 4/45
72/72 ━━━━━━━━━━━━━━━━━━━━ 3s 45ms/step - loss: 0.0011 - val_loss: 7.8507e-04
Epoch 5/45
72/72 ━━━━━━━━━━━━━━━━━━━━ 3s 45ms/step - loss: 0.0013 - val_loss: 7.4785e-04
Epoch 6/45
72/72 ━━━━━━━━━━━━━━━━━━━━ 3s 46ms/step - loss: 0.0011 - val_loss: 7.8069e-04
Epoch 7/45
72/72 ━━━━━━━━━━━━━━━━━━━━ 3s 45ms/step - loss: 0.0010 - val_loss: 7.9095e-04
Epoch 8/45
72/72 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step - loss: 8.7121e-04 - val_loss: 7.2867e-04
Epoch 9/45
72/72 ━━━━━━━━━━━━━━━━━━━━ 3s 46ms/step - loss: 9.7240e-04 - val_loss: 9.9583e-04
Epoch 10/45
72/72 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step - loss: 8.7639e-04 - val_loss: 6.9190e-04
Epoch 11/45
72/72 ━━━━━━━━━━━━━━━━━━━━ 4s 49ms/step - loss: 9.0032e-04 - val_loss: 5.9967e-04
Epoch 12/45
72/72 ━━━━

In [73]:
# Crear dataset de prueba
test_data = scaled_data[training_data_len - window_size: , :]

X_test = []
y_test = data[training_data_len:, :] # Estos son los valores reales (sin escalar)

for i in range(window_size, len(test_data)):
    X_test.append(test_data[i-window_size:i, 0])

X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Realizar predicciones
predictions = model.predict(X_test)

# Invertir el escalado para obtener precios reales
predictions = scaler.inverse_transform(predictions)

21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step


In [74]:
# Calcular métricas
rmse = np.sqrt(mean_squared_error(y_test, predictions))
mae = mean_absolute_error(y_test, predictions)
mape = mean_absolute_percentage_error(y_test, predictions)

print("--------------------------------------------------")
print("Métricas de Evaluación del Modelo:")
print(f"RMSE (Root Mean Squared Error): {rmse:.4f}")
print(f"MAE  (Mean Absolute Error):     {mae:.4f}")
print(f"MAPE (Mean Absolute % Error):   {mape:.2%}")
print("--------------------------------------------------")

--------------------------------------------------
Métricas de Evaluación del Modelo:
RMSE (Root Mean Squared Error): 5.6788
MAE  (Mean Absolute Error):     4.1925
MAPE (Mean Absolute % Error):   2.21%
--------------------------------------------------


# Análisis de Resultados:

- **RMSE (~5.67):** Indica que, en promedio, el modelo se desvía unos 5 dólares del precio real. Dado que la acción cotiza por encima de los 200 USD, es un error aceptable.

- **MAPE (~2.21%):** El error porcentual es bajo, esto nos dice que el modelo tiene una alta capacidad predictiva y generaliza bien con datos no vistos.

- **Gráfica (abajo):** Se observa que la línea roja (predicción) sigue muy de cerca la tendencia de la línea azul (real), aunque con un ligero retraso (lag), comportamiento típico de los modelos LSTM en finanzas.

In [75]:
# Grafica de resultados
train = df[:training_data_len]
valid = df[training_data_len:]
valid['Predictions'] = predictions

fig = go.Figure()
fig.add_trace(go.Scatter(x=train['Date'], y=train['Close'], mode='lines', name='Entrenamiento', line=dict(color='gray')))
fig.add_trace(go.Scatter(x=valid['Date'], y=valid['Close'], mode='lines', name='Valor Real', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=valid['Date'], y=valid['Predictions'], mode='lines', name='Predicción', line=dict(color='red')))
fig.update_layout(title='Evaluación: Real vs Predicción', template='plotly_dark')
fig.show()

C:\Users\canal\AppData\Local\Temp\ipykernel_11936\1673137633.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [76]:
# Pronóstico a 7 días
days_to_predict = 7
future_predictions = []
current_batch = scaled_data[-window_size:].reshape((1, window_size, 1))

for i in range(days_to_predict):
    current_pred = model.predict(current_batch, verbose=0)
    future_predictions.append(current_pred[0])
    current_batch = np.append(current_batch[:, 1:, :], [current_pred], axis=1)

future_predictions = scaler.inverse_transform(future_predictions)

# Fechas futuras
last_date = df['Date'].iloc[-1]
future_dates = []
current_date = last_date
while len(future_dates) < days_to_predict:
    current_date += timedelta(days=1)
    if current_date.weekday() < 5: 
        future_dates.append(current_date)

df_future = pd.DataFrame({'Date': future_dates, 'Prediction': future_predictions.flatten()})

# Gráfica Final
fig_full = go.Figure()
fig_full.add_trace(go.Scatter(x=df.tail(100)['Date'], y=df.tail(100)['Close'], mode='lines', name='Historia Reciente', line=dict(color='blue')))
fig_full.add_trace(go.Scatter(x=df_future['Date'], y=df_future['Prediction'], mode='lines+markers', name='Pronóstico 7 días', line=dict(color='red', width=2, dash='dot')))
fig_full.update_layout(title=f'Pronóstico a {days_to_predict} días para TTWO', template='plotly_dark')
fig_full.show()

print(df_future)

        Date  Prediction
0 2025-12-01  235.936266
1 2025-12-02  235.963653
2 2025-12-03  234.552591
3 2025-12-04  232.392845
4 2025-12-05  229.932077
5 2025-12-08  227.397100
6 2025-12-09  224.892112


# Conclusiones del Proyecto:

Se observa que la predicción a 7 días pierde precisión conforme se aleja en el tiempo debido a la acumulación de errores en la predicción recursiva. Para un entorno productivo real, seria mejor utilizar este modelo para predicciones diarias o modificar la arquitectura de salida para predecir el vector de 7 días simultáneamente.

1. **Desempeño:** El modelo LSTM demostró ser robusto con un MAPE inferior al 3%. Logró capturar la tendencia general de las acciones de TTWO.

2. **Limitaciones:** Al ser un modelo univariado (solo usa el precio pasado), no considera factores externos como noticias del sector, lanzamientos de juegos (ej. GTA VI) o reportes trimestrales, que causan picos repentinos.

3. **Pronóstico:** La proyección a 7 días sugiere una tendencia bajista con un precio objetivo de cierre aproximado de 224.90 USD.

4. **Mejoras Futuras:** Se podría mejorar incluyendo análisis de sentimiento de noticias o variables macroeconómicas (S&P 500) como features adicionales.